In [ ]:
# Fractional Delay Line

from scipy import signal
from scipy.fftpack import fft, ifft
import numpy as np

from IPython.display import Audio 
import soundfile as sf

maxDelay = 3000
x, Fs = sf.read("hw2_audio.wav")
returnBuffer = np.zeros(len(x))
circularBuffer = np.zeros(maxDelay)
writeIndex = 0
readIndex = 0
# generate sine wave
f = 440
t = np.linspace(0, 1, Fs)
sineWave = np.sin(2 * np.pi * f * t)
sampleRate = 44100

def fractional_index(signal, idx):
    i = int(np.floor(idx))        # integer part
    frac = idx - i                # fractional part (0 <= frac < 1)

    # Boundary checks
    if i < 0:
        return 0.0
    if i >= len(signal):
        return 0.0
    
    # Next index
    i_next = i + 1
    
    # If the next index is out of range, just return the sample at i
    if i_next >= len(signal):
        return float(signal[i])
    
    # Linear interpolation
    val_i     = float(signal[i])
    val_i_next= float(signal[i_next])
    return (1.0 - frac)*val_i + frac*val_i_next


def fractional_delay_with_feedback_modulation(
    x, 
    fs, 
    fb=0.7,       # feedback coefficient
    g=0.9,        # overall gain applied to the input portion
    M_min=10.0,   # minimum delay in samples
    M_max=30.0,   # maximum delay in samples
    f_mod=1.0     # modulation frequency in Hz
):

    N = len(x)
    y = np.zeros(N, dtype=np.float32)

    n = np.arange(N)
    M_amp  = 0.5 * (M_max - M_min)
    M_dc   = 0.5 * (M_max + M_min)
    M_array = M_dc + M_amp * np.sin(2.0 * np.pi * f_mod * n / fs)

    for i in range(N):
        M = M_array[i]
        x_delayed = fractional_index(x, i - M)
        y_delayed = fractional_index(y, i - M)

        # Apply difference equation
        y[i] = fb * y_delayed + g * x_delayed

    return y

returnAudio = fractional_delay_with_feedback_modulation(x, Fs)
Audio(returnAudio, rate=Fs)



Theory:

1. H(z) = (1 - z^-1)^2/(1 - (-.261764 + j(.5473)* z^-1)*(1 - (-.261764 - j(.5473)*z^-1)))
2. Zeros: z = 1, poles: -0.262 +- j*.547